In [ ]:
Evaluates the human consensus and merges files and adds descriptive columns
Requires blinded human determination from panel

In [4]:
version

               _                           
platform       x86_64-apple-darwin17.0     
arch           x86_64                      
os             darwin17.0                  
system         x86_64, darwin17.0          
status                                     
major          4                           
minor          2.2                         
year           2022                        
month          10                          
day            31                          
svn rev        83211                       
language       R                           
version.string R version 4.2.2 (2022-10-31)
nickname       Innocent and Trusting       

In [1]:
library(plyr)
library(tidyverse)
library(readxl)
library(ggpubr)

── Attaching core tidyverse packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::arrange()   masks plyr::arrange()
✖ purrr::compact()   masks plyr::compact()
✖ dplyr::count()     masks plyr::count()
✖ dplyr::desc()      masks plyr::desc()
✖ dplyr::failwith()  masks plyr::failwith()
✖ dplyr::filter()    masks stats::filter()
✖ dplyr::id()        masks plyr::id()
✖ dplyr::lag()       masks stats::lag()
✖ dplyr::mutate()    masks plyr::mutate()
✖ dplyr::rename()    masks plyr::rename()
✖ dplyr::summarise() masks plyr::summarise()
✖ dplyr::summarize()

In [2]:
`%notin%` <- Negate(`%in%`)

In [ ]:
runVersion = "test"

In [ ]:
MSigDB_unmaskingDict <- c("Name A"="LLM", "Name B"= "GO", "Neither" = "Neither", "Tied" = "Tied")

In [ ]:
NeST_unmaskingDict <- c("Name B"="LLM", "Name A"= "GO",  "Neither" = "Neither", "Tied" = "Tied")

In [ ]:
## read in answers from google sheet - human input

In [ ]:
getHumanConsensus = function(Winner_human1,Winner_human2,  Winner_human3, OverridingDecision = NaN ){
  # dataframe row 
  if (!is.na(OverridingDecision)){
    humanConsensus = OverridingDecision
  } else {
    winnerList =  c(Winner_human1, Winner_human2,Winner_human3 )
    sortedCounts = sort(table(winnerList), decreasing = TRUE)
    if  (sortedCounts[1] ==1){
      # all are tied
        if (("Tied" %in% winnerList) & ("Neither" %in% winnerList) ){
            humanConsensus = "Neither" # Neither is stronger than Tied
      } else if ("Tied" %in% winnerList){
        humanConsensus = "Tied"
      } else if ("Neither" %in% winnerList){
        humanConsensus = "Neither"
      } else {
        humanConsensus = "need help"
      }
      
    } else {
      
   # print(sortedCounts)
      humanConsensus = names(sortedCounts[1] )
      
    }
  }
  
  return(humanConsensus)
}


In [ ]:
getSetSize = function(genesStr){
    geneList = str_split(string = genesStr, pattern = "[, ]")[[1]]
    return(length(geneList))
}

#### NeST

In [ ]:
LLM_human_NeST = read_xlsx(path = 'data/NeST blinded.xlsx', sheet = 'Sheet1')


In [ ]:
LLM_human_NeST = LLM_human_NeST %>% 
mutate(DataSet = "NeST") %>%
rowwise() %>%
mutate(humanConsensusMasked = getHumanConsensus(Winner_human1,Winner_human2,  Winner_human3, OverridingDecision )) %>%
mutate(humanConsensus = NeST_unmaskingDict[ humanConsensusMasked])

In [ ]:
table(LLM_human_NeST$humanConsensus)

#### MSigDB

In [ ]:
LLM_human_MSigDB = read_xlsx(path = 'data/MSigDB blinded.xlsx', sheet = 'Sheet1') 


In [ ]:
dim(LLM_human_MSigDB)

In [ ]:
LLM_human_MSigDB = LLM_human_MSigDB %>% 
mutate(DataSet = "MSigDB") %>%
rowwise() %>%
mutate(humanConsensusMasked = getHumanConsensus(Winner_human1,Winner_human2,  Winner_human3 )) %>%
mutate(humanConsensus = MSigDB_unmaskingDict[ humanConsensusMasked])

In [ ]:
table(LLM_human_MSigDB$humanConsensus)

runVersion### Read in the semantic analyses

In [ ]:
runVersion

In [ ]:
MSigDB_table_sub_LLM_Enrichr_simVals_DF = read_delim("data/MSigDB_table_subset_LLM_Enrichr_simVals_refs_DF.txt", delim = "\t")


In [ ]:
if (runVersion == "test"){
    MSigDB_table_sub_LLM_Enrichr_simVals_DF = read_delim("data/MSigDB_table_subset_LLM_Enrichr_simVals_refs_DF.txt", delim = "\t")
} else {
    MSigDB_table_sub_LLM_Enrichr_simVals_DF = read_delim("data/MSigDB_table_LLM_Enrichr_simVals_refs_DF.txt", delim = "\t")
    }

In [ ]:
MSigDB_table_sub_LLM_Enrichr_simVals_DF = rename(MSigDB_table_sub_LLM_Enrichr_simVals_DF, 
                                               c("Human Name" ="Name",
                                                "Enrichr genes" = "Genes.1"))

In [ ]:
MSigDB_table_sub_LLM_Enrichr_simVals_DF = MSigDB_table_sub_LLM_Enrichr_simVals_DF %>%
rowwise() %>%
mutate(n_Genes = getSetSize(`Genes`),
      log2_n_Genes = log2(n_Genes))

#### NeST

In [ ]:
if (runVersion == "test"){
    NeST_table_sub_LLM_Enrichr_simVals_DF = read_delim("data/NeST_table_subset_LLM_Enrichr_simVals_refs_DF.txt", delim = "\t")
} else {
    NeST_table_sub_LLM_Enrichr_simVals_DF = read_delim("data/NeST_table_LLM_Enrichr_simVals_refs_DF.txt", delim = "\t")
    }

In [ ]:
NeST_table_sub_LLM_Enrichr_simVals_DF = rename(NeST_table_sub_LLM_Enrichr_simVals_DF, 
                                            c( "Enrichr genes" = "Genes.1"))

In [ ]:
NeST_table_sub_LLM_Enrichr_simVals_DF = NeST_table_sub_LLM_Enrichr_simVals_DF %>%
rowwise() %>%
mutate(n_Genes = getSetSize(`Genes`),
      log2_n_Genes = log2(n_Genes))

### Join the dataframe together

In [ ]:
NeST_analysis_DF = inner_join(x = NeST_table_sub_LLM_Enrichr_simVals_DF[,c("NEST ID", "Genes", "LLM Name", "LLM Analysis", "References",
                                      "GO term", "GO ID", "Enrichr genes", "Overlap", "P-value", "Adjusted P-value", 
                                      "LLM_name_human_name_sim", "GO_term_human_name_sim", "winner", 
                                      "n_Genes", "log2_n_Genes")],
                              y = LLM_human_NeST[,c("NEST ID", "Human Name", "humanConsensus", "OverridingDecision", "DataSet")],
                              by = "NEST ID")
                                  

In [ ]:
MSigDB_analysis_DF = inner_join(x = MSigDB_table_sub_LLM_Enrichr_simVals_DF[,c("Human Name", "Genes", "LLM Name", "LLM Analysis", "References",
                                      "GO term", "GO ID", "Enrichr genes", "Overlap", "P-value", "Adjusted P-value", 
                                      "LLM_name_human_name_sim", "GO_term_human_name_sim", "winner", 
                                      "n_Genes", "log2_n_Genes")],
                              y = LLM_human_MSigDB[,c("Human Name", "humanConsensus", "DataSet")],
                                 by =  "Human Name")
                                  

In [ ]:
merged_analysis_DF = rbind.fill(NeST_analysis_DF,MSigDB_analysis_DF)

In [ ]:
merged_analysis_DF = merged_analysis_DF %>%
rowwise() %>%
mutate(log2_rat_LLM_GO = log2(LLM_name_human_name_sim/GO_term_human_name_sim),
       diff_LLM_GO = LLM_name_human_name_sim - GO_term_human_name_sim)

In [ ]:
merged_analysis_DF$humanConsensus <- factor(merged_analysis_DF$humanConsensus, levels=c("LLM", "GO", "Tied", "Neither"))


In [ ]:
merged_analysis_DF$winner <- factor(merged_analysis_DF$winner, levels=c("LLM", "GO", "Tied", "Neither"))


In [ ]:
if (runVersion == "test"){
    mergedFile = "data/merged_subset_LLM_Enrichr_simVals_refs.txt"
} else {
    mergedFile = "data/merged_LLM_Enrichr_simVals_refs.txt"
}

    
    


In [ ]:
write_delim(x = merged_analysis_DF[,c('DataSet','Human Name', 'NEST ID', 'Genes', 'n_Genes', 'log2_n_Genes',
                                     'GO term','GO ID','Enrichr genes', 'Overlap','P-value','Adjusted P-value',
                                     'LLM Name','LLM Analysis','References',
                                      'LLM_name_human_name_sim', 'GO_term_human_name_sim',
                                      'winner','humanConsensus', 'OverridingDecision')],
            file = mergedFile, delim = "\t")


In [ ]:
merged_analysis_DF$DataSet

### Get percentage of genes used in each of LLM and Enrichr

In [ ]:
library(comprehenr)


In [ ]:
getEnrichrGeneCount = function(systemGenes, Enrichr_Genes){
    systemGenesList = str_split(string = systemGenes, pattern = " ")[[1]];
    Enrichr_GenesList = str_split(string = Enrichr_Genes, pattern = ";")[[1]];
    return(length(intersect(systemGenesList, Enrichr_GenesList)))
    }
    

In [ ]:
getLLMGeneCount  = function(GenesFixed, LLM_Analysis){
    systemGenesList = str_split(string = GenesFixed, pattern = " ")[[1]];
    
    n_LLMGenes = to_vec(for(systemGene in systemGenesList) str_detect(string = LLM_Analysis, pattern = systemGene)) %>%
           sum()
        
    return(n_LLMGenes)


    }

In [ ]:
merged_analysis_DF = merged_analysis_DF %>% 
rowwise() %>%
mutate(GenesFixed = str_replace_all(string = `Genes`, pattern = ",", replacement = " "),
      n_overlap_Enrichr = getEnrichrGeneCount(`GenesFixed`, `Enrichr genes`),
      perc_overlap_Enrichr = 100*(n_overlap_Enrichr/n_Genes),
      n_overlap_LLM = getLLMGeneCount(`GenesFixed`, `LLM Analysis`),
      perc_overlap_LLM = 100*(n_overlap_LLM/n_Genes))
  

In [ ]:
write_delim(x = merged_analysis_DF[,c('DataSet','Human Name', 'NEST ID', 'GenesFixed', 'n_Genes', 'log2_n_Genes',
                                     'GO term','GO ID','Enrichr genes', 'n_overlap_Enrichr', 'perc_overlap_Enrichr','P-value','Adjusted P-value',
                                     'LLM Name','LLM Analysis','n_overlap_LLM','perc_overlap_LLM', 'References',
                                      'LLM_name_human_name_sim', 'GO_term_human_name_sim',
                                      'winner','humanConsensus', 'OverridingDecision')],
            file = mergedFile, delim = "\t")